In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/benofben/vertex-ai-samples/blob/master/notebooks/community/neo4j/graph_paysim.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/benofben/vertex-ai-samples/tree/master/notebooks/community/neo4j/graph_paysim.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

# Overview
This notebook is an example of using Neo4j with Vertex AI.  It takes PaySim data from a Neo4j database, then puts that into Vertex AI Feature Store.  It  also runs a classification on the data.

## Dataset
The notebook uses a version of the PaySim dataset that has been modified to work with Neo4j's graph database.  PaySim is a synthetic fraud dataset.  The goal is to identify whether or not a given transaction constitutes fraud.  The [original version of the dataset](https://github.com/EdgarLopezPhD/PaySim) has tabular data.

Neo4j has worked on a modified version that generates a graph dataset [here](https://github.com/voutilad/PaySim).  We've pregenerated a copy of that dataset that you can grab [here](https://storage.googleapis.com/neo4j-datasets/paysim.dump).  You'll want to download that dataset and then upload it to Neo4j AuraDS.  AuraDS is a graph data science tool that is offered as a service on GCP.  Instructions on signing up and uploading the dataset are available [here](https://github.com/neo4j-partners/aurads-paysim).

##Objective
In this notebook, you will learn how to use Neo4j AuraDS to create graph features.  You'll then use those new features to solve a classification problem with Vertex AI.


##Costs
This tutorial uses billable components of Google Cloud:

* Cloud Storage
* Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

##Set up your development environment
We suggest you use Colab for this notebook.

## Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

## Install additional Packages
We assume that you've already loaded the PaySim data into a Neo4j instance and have the credentials to connect to that.

You'll also need to install a few packages.

In [1]:
!pip install neo4j

     |████████████████████████████████| 89 kB 2.5 MB/s 
  Created wheel for neo4j: filename=neo4j-4.4.0-py3-none-any.whl size=114860 sha256=e7cb25181483e669711142c351931aeb8ae22b47999aece36b479358125b6a0d
  Stored in directory: /root/.cache/pip/wheels/b8/cb/e4/d34e10b40d33911c281a6e9aa038d54ca7435310529b3f6042
Successfully built neo4j


In [2]:
!pip install --upgrade google-cloud-storage google.cloud.aiplatform

     |████████████████████████████████| 105 kB 5.1 MB/s 
     |████████████████████████████████| 1.6 MB 40.8 MB/s 
     |████████████████████████████████| 95 kB 4.2 MB/s 
     |████████████████████████████████| 75 kB 3.5 MB/s 
     |████████████████████████████████| 45 kB 3.4 MB/s 
     |████████████████████████████████| 203 kB 67.6 MB/s 
     |████████████████████████████████| 1.1 MB 57.7 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.26.3
    Uninstalling google-api-core-1.26.3:
      Successfully uninstalled google-api-core-1.26.3
  Attempting uninstall: google-resumable-media
    Found existing installation: google-resumable-media 0.4.1
    Uninstalling google-resumable-media-0.4.1:
      Successfully uninstalled google-resumable-media-0.4.1
  Attempting uninstall: google-

##Restart the kernel
After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

## Working with Neo4j
In this section we're going to connect to Neo4j and look around the database.  We're going to generate some new features in the dataset using Neo4j's Graph Data Science library.  Finally, we'll load the data into a Pandas dataframe so that it's all ready to put into GCP Feature Store.

You'll need to enter the credentials for your Aura DS instance below.  Some example credentials are shown below.  You can get your credentials by following this [walkthrough](https://github.com/neo4j-partners/aurads-paysim).

The "DB_NAME" is always neo4j for AuraDS.  It is different from the name you gave your database tenant in the AuraDS console.

In [12]:
DB_ULR = 'neo4j+s://df9cad2b.databases.neo4j.io'
DB_USER = 'neo4j'
DB_PASS = 'password'
DB_NAME = 'neo4j'

In [13]:
import pandas as pd
from neo4j import GraphDatabase

In [14]:
driver = GraphDatabase.driver(DB_ULR, auth=(DB_USER, DB_PASS))

Now, let's explore the data in the database a bit to understand what we have to work with.

In [15]:
# node labels
with driver.session(database = DB_NAME) as session:
  result = session.read_transaction( lambda tx: 
    tx.run(
    """
    CALL db.labels() YIELD label
    CALL apoc.cypher.run('MATCH (:`'+label+'`) RETURN count(*) as freq', {})
    YIELD value
    RETURN label, value.freq AS freq
    """
    ).data()
  )
  df = pd.DataFrame(result)
  display(df)

,label,freq
0,Node,0
1,Client,11270
2,Bank,5
3,Merchant,3465
4,Mule,0
5,CashIn,746751
6,CashOut,424574
7,Debit,130284
8,Payment,542443
9,Transfer,0


In [16]:
# relationship types
with driver.session(database = DB_NAME) as session:
  result = session.read_transaction( lambda tx: 
    tx.run(
      """
      CALL db.relationshipTypes() YIELD relationshipType as type
      CALL apoc.cypher.run('MATCH ()-[:`'+type+'`]->() RETURN count(*) as freq', {})
      YIELD value
      RETURN type AS relationshipType, value.freq AS freq
      ORDER by freq DESC
      """
      ).data()
    )
df = pd.DataFrame(result)
display(df)

,relationshipType,freq
0,PERFORMED,1844052
1,TO,1844052
2,NEXT,1833720
3,HAS_SSN,11330
4,HAS_EMAIL,11330
5,HAS_PHONE,11330
6,FIRST_TX,10332
7,LAST_TX,10332


In [17]:
# transaction types
with driver.session(database = DB_NAME) as session:
  result = session.read_transaction( lambda tx: 
    tx.run(
    """
    MATCH (t:Transaction)
    WITH sum(t.amount) AS globalSum, count(t) AS globalCnt
    WITH *, 10^3 AS scaleFactor
    UNWIND ['CashIn', 'CashOut', 'Payment', 'Debit', 'Transfer'] AS txType
      CALL apoc.cypher.run('MATCH (t:' + txType + ')
        RETURN sum(t.amount) as txAmount, count(t) AS txCnt', {})
      YIELD value
    RETURN txType,value.txAmount AS TotalMarketValue
    """
    ).data()
  )
  df = pd.DataFrame(result)
  display(df)

,txType,TotalMarketValue
0,CashIn,1.040582e+11
1,CashOut,5.385410e+10
2,Payment,9.646814e+10
3,Debit,1.016829e+09
4,Transfer,0.000000e+00


## Create a New Feature with a Graph Embedding using Neo4j
First we're going to create an in memory graph represtation of the data in Neo4j Graph Data Science (GDS).

Note, if you get an error saying the graph already exists, that's probably because you ran this code before.  You can destroy it using the command in the cleanup section of this notebook.

In [18]:
with driver.session(database = DB_NAME) as session:
  result = session.read_transaction( lambda tx: 
    tx.run(
    """
    CALL gds.graph.create.cypher('client_graph', 
      'MATCH (c:Client) RETURN id(c) as id, c.num_transactions as num_transactions, c.total_transaction_amnt as total_transaction_amnt, c.is_fraudster as is_fraudster',
      'MATCH (c:Client)-[:PERFORMED]->(t:Transaction)-[:TO]->(c2:Client) return id(c) as source, id(c2) as target, sum(t.amount) as amount, "TRANSACTED_WITH" as type ')
    """
    ).data()
  )
  df = pd.DataFrame(result)
  display(df)

,nodeQuery,relationshipQuery,graphName,nodeCount,relationshipCount,createMillis
0,"MATCH (c:Client) RETURN id(c) as id, c.num_tra...",MATCH (c:Client)-[:PERFORMED]->(t:Transaction)...,client_graph,11270,26035,875


Now we can generate an embedding from that graph.  This is a new feature we can use in our predictions.  We're using FastRP, which is a more full featured and higher performance of Node2Vec.  You can learn more about that [here](https://neo4j.com/docs/graph-data-science/current/algorithms/fastrp/).

In [19]:
with driver.session(database = DB_NAME) as session:
  result = session.read_transaction( lambda tx: 
    tx.run(
    """
    CALL gds.fastRP.mutate('client_graph',{
      relationshipWeightProperty:'amount',
      iterationWeights: [0.0, 1.00, 1.00, 0.80, 0.60],
      featureProperties: ['num_transactions', 'total_transaction_amnt'],
      propertyRatio: .25, 
      embeddingDimension: 16,
      randomSeed: 1, 
      mutateProperty:'embedding'
    })
    """
    ).data()
  )
  df = pd.DataFrame(result)
  display(df)

,nodePropertiesWritten,mutateMillis,nodeCount,createMillis,computeMillis,configuration
0,11270,1,11270,1,85,"{'nodeSelfInfluence': 0, 'relationshipWeightPr..."


Finally we dump that out to a dataframe

In [20]:
with driver.session(database = DB_NAME) as session:
  result = session.read_transaction( lambda tx: 
    tx.run(
    """
    CALL gds.graph.streamNodeProperties
    ('client_graph', ['embedding', 'num_transactions', 'total_transaction_amnt', 'is_fraudster'])
    YIELD nodeId, nodeProperty, propertyValue
    RETURN nodeId, nodeProperty, propertyValue
    """
    ).data()
  )
  df = pd.DataFrame(result)
  df.head()

Now we need to take that dataframe and shape it into something that better represents our classification problem.

In [21]:
x = df.pivot(index='nodeId', columns='nodeProperty', values='propertyValue')
x = x.reset_index()
x.columns.name = None
x.head()

,nodeId,embedding,is_fraudster,num_transactions,total_transaction_amnt
0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,4,118919
1,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-9223372036854775808,0,0
2,5,"[-4.998395475297457e-09, 5.79870196304455e-09,...",1,80,7.48446e+06
3,8,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-9223372036854775808,0,0
4,10,"[0.02352502942085266, -0.023524967953562737, 2...",1,227,3.75806e+07


is_fraudster will have a value of 0 or 1 if populated.  If the value is 10000 then it's unlabled, so we're going to drop it.

In [22]:
x = x.loc[x['is_fraudster'] != 10000]
x.head()

,nodeId,embedding,is_fraudster,num_transactions,total_transaction_amnt
0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,4,118919
1,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-9223372036854775808,0,0
2,5,"[-4.998395475297457e-09, 5.79870196304455e-09,...",1,80,7.48446e+06
3,8,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-9223372036854775808,0,0
4,10,"[0.02352502942085266, -0.023524967953562737, 2...",1,227,3.75806e+07


Note that the embedding row is an array.  To make this dataset more consumable, we should flatten that out into multiple individual features: embedding_0, embedding_1, ... embedding_n.

In [23]:
embeddings = pd.DataFrame(x['embedding'].values.tolist()).add_prefix('embedding_')
merged = x.drop(columns=['embedding']).merge(embeddings, left_index=True, right_index=True)
features_df = merged.drop(columns=['is_fraudster', 'num_transactions', 'total_transaction_amnt'])
train_df = merged.drop(columns=['nodeId'])

FEATURES_FILENAME = 'features.csv'
features_df.to_csv(FEATURES_FILENAME, index=False)

This dataset is too small to use with Vertex AI AutoML Tables. For sake of demonstration, we're going to repeat it a few times. Don't do this in the real world.

In [24]:
TRAINING_FILENAME = 'train.csv'
pd.concat([train_df for i in range(10)]).to_csv(TRAINING_FILENAME, index=False)

And that's it!  The dataframe now has a nice dataset that we can use with GCP Vertex AI.

## Authenticate your Google Cloud account


In [29]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

## Setup GCP Environmental Variables

In [32]:
import os
PROJECT_ID = 'neo4jbusinessdev'
os.environ['GCLOUD_PROJECT'] = PROJECT_ID

##Upload to a GCP Cloud Storage Bucket

To get the data into Vertex AI, we must first put it in a bucket as a CSV.

In [38]:
from google.cloud import storage
client = storage.Client()

In [39]:
# Create a bucket
STORAGE_BUCKET = 'paysim'
bucket = client.bucket(STORAGE_BUCKET)
client.create_bucket(bucket)

<Bucket: paysim>

In [40]:
# Upload our files to that bucket
for filename in [FEATURES_FILENAME, TRAINING_FILENAME]:
  blob = bucket.blob(filename)
  blob.upload_from_filename(filename)

## Create a Dataset with Vertex AI
Now we're going to take the CSV in our bucket and turn that into a dataset in Vertex AI

In [41]:
from google.cloud import aiplatform
aiplatform.init()

dataset = aiplatform.TabularDataset.create(display_name='paysim', gcs_source=os.path.join('gs://', STORAGE_BUCKET, TRAINING_FILENAME))
dataset.wait()

print(f'\tDataset: "{dataset.display_name}"')
print(f'\tname: "{dataset.resource_name}"')

	Dataset: "paysim"
	name: "projects/803648085855/locations/us-central1/datasets/6086254256577314816"


In [46]:
# TODO: don't hard code the dimension size
EMBEDDING_DIMENSION = 16
embedding_column_names = [ 'embedding_{}'.format(i) for i in range(EMBEDDING_DIMENSION) ]
other_column_names = ['num_transactions', 'total_dollar_amnt']
all_columns = other_column_names + embedding_column_names

column_specs = [
  {column: 'numeric'} for column in all_columns
]

In [47]:
job = aiplatform.AutoMLTabularTrainingJob(
    display_name = 'train-paysim-automl-1',
    optimization_prediction_type = 'classification',
    column_specs = column_specs,
)

AttributeError: ignored

In [ ]:
model = job.run(
    dataset = dataset,
    target_column = 'is_fraudster',
    training_fraction_split = 0.8,
    validation_fraction_split = 0.1,
    test_fraction_split = 0.1,
    model_display_name = 'paysim-prediction-model',
    disable_early_stopping = False,
    budget_milli_node_hours = int(1000 / 60 * 5) # Limit to 5 minute running time
)

## Classification with Vertex AI
In this section, we're going to run two classifiers and compare results.  The first will use the standard PaySim features.  The second will use our new graph features.

## Loading Data into GCP Feature Store
In this section, we'll take our dataframe with newly engineered features and load that into GCP feature store.

In [48]:
REGION = 'us-central1'
FEATURESTORE_ID = 'paysim'
ENTITY_NAME = 'payer'

In [49]:
from google.cloud.aiplatform_v1 import FeaturestoreServiceClient
api_endpoint = '{}-aiplatform.googleapis.com'.format(REGION)
fs_client = FeaturestoreServiceClient(client_options={'api_endpoint': api_endpoint})

resource_path = fs_client.common_location_path(PROJECT_ID, REGION)
fs_path = fs_client.featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID)
entity_path = fs_client.entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITY_NAME)

First, let's check if the Feature Store already exists

In [50]:
from grpc import StatusCode
def check_has_resource(callable):
  has_resource = False
  try:
    callable()
    has_resource = True
  except Exception as e:
    if not hasattr(e, 'grpc_status_code') or e.grpc_status_code != StatusCode.NOT_FOUND:
      raise e
  return has_resource

In [51]:
feature_store_exists = check_has_resource(
    lambda: fs_client.get_featurestore(
      name= fs_path
    )
)

In [52]:
from google.cloud.aiplatform_v1.types import featurestore_service as featurestore_service_pb2
from google.cloud.aiplatform_v1.types import featurestore as featurestore_pb2
from google.cloud.aiplatform_v1.types import feature as feature_pb2
from google.cloud.aiplatform_v1.types import entity_type as entity_type_pb2
from google.cloud.aiplatform_v1.types import io as io_pb2

if not feature_store_exists:
  create_lro = admin_client.create_featurestore(
      featurestore_service_pb2.CreateFeaturestoreRequest(
          parent=resource_path,
          featurestore_id=FEATURESTORE_ID,
          featurestore=featurestore_pb2.Featurestore(
              online_serving_config=featurestore_pb2.Featurestore.OnlineServingConfig(
                  fixed_node_count=1
              ),
          ),
      )
  )

  print(create_lro.result())

NameError: ignored

In [ ]:
entity_type_exists = check_has_resource(    
    lambda: fs_client.get_entity_type(
        name=entity_path
    )
)

if not entity_type_exists:
  users_entity_type_lro = fs_client.create_entity_type(
    featurestore_service_pb2.CreateEntityTypeRequest(
        parent=fs_path,
        entity_type_id=ENTITY_NAME,
        entity_type=entity_type_pb2.EntityType(
            description="Main entity type",
        ),
      )
  )
  print(users_entity_type_lro.result())

  feature_requests = [
    featurestore_service_pb2.CreateFeatureRequest(
      feature=feature_pb2.Feature(
          value_type=feature_pb2.Feature.ValueType.DOUBLE,
          description="Embedding {} from Neo4j".format(i),
      ),
      feature_id="embedding_{}".format(i),
    )
    for i in range(EMBEDDING_DIMENSION)
  ]
  create_features_lro = fs_client.batch_create_features(
      parent=entity_path,
      requests=feature_requests,
  )
  print(create_features_lro.result())

In [ ]:
feature_specs = [
  featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="embedding_{}".format(i))
  for i in range(EMBEDDING_DIMENSION)         
]

from google.protobuf.timestamp_pb2 import Timestamp

import_request = fs_client.import_feature_values(
    featurestore_service_pb2.ImportFeatureValuesRequest(
      entity_type=entity_path,
      csv_source=io_pb2.CsvSource(
          gcs_source=io_pb2.GcsSource(
              uris=[
                  os.path.join('gs://', STORAGE_BUCKET, FEATURES_FILENAME)
              ]
          )
      ),
      entity_id_field="nodeId",
      feature_specs=feature_specs,
      worker_count=1,
      feature_time=Timestamp().GetCurrentTime()
  )
)

print(import_request.result())

##Cleanup

To delete the Graph Data Science representation of the graph, run this:

In [ ]:
with driver.session(database = DB_NAME) as session:
    result = session.read_transaction( lambda tx: 
        tx.run(
        """
        CALL gds.graph.drop('client_graph')
        """
        ).data()
    )